In [1]:
#!pip install textblob

### Краткое изложение статьи

В статье описывается способ предсказать движение цены на основе фундментального анализа, а точнее использование сентиментаческого анализа новостей и технических индикаторов по типу скользящей средней и объема.

Так же делается выбор предскзания между регрессионным или классикацией оценки цены актива, и выбирается в пользу классификации, так как его предсказывать проще и больше возможностей для торговых идей. Соответственно таргет, что предсказываем - это цена закрытия (а точнее adj_close тк она уже скорректирована на сплит акций актива), которая показывает изменения в +% или -% в зависимости от прошлого дня.

Поэтому предсказываем 1, если цена выросла или не изменилась, и 0, если пошла вниз.

Тестируются различные модели сентимент анализа и модели машинного обучения, и были отобраны наилучшие
это метод опорных векторов - модель классификации (SVC) и TextBlob для сентимент анализа.

#1 Выгрузим данные и предобработаем их, новости и котировки начинают мэтчитьтся с 2016-01-04, соотвественно с этой даты и будем брать оба датасета

In [2]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [3]:
sp500 = pd.read_excel('yahoo_sp500.xls')
sp500['Date'] = pd.to_datetime(sp500['Date']).dt.strftime('%Y-%m-%d')
sp500 = sp500.sort_values(by=['Date'])
sp500 = sp500.query('Date >= "2016-01-01"').reset_index(drop=True)
sp500.columns = sp500.columns.str.strip('*').str.lower()
sp500 = sp500[['date','volume','adj close']]
sp500 = sp500.rename(columns={'adj close':'adj_close'})
sp500

,date,volume,adj_close
0,2016-01-04,4304880000,2012.66
1,2016-01-05,3706620000,2016.71
2,2016-01-06,4336660000,1990.26
3,2016-01-07,5076590000,1943.09
4,2016-01-08,4664940000,1922.03
...,...,...,...
1022,2020-01-27,3823100000,3243.63
1023,2020-01-28,3526720000,3276.24
1024,2020-01-29,3584500000,3273.40
1025,2020-01-30,3787250000,3283.66


In [4]:
%%time

nws = pd.read_csv('news.csv')
nws['date'] = pd.to_datetime(nws['date']).dt.strftime('%Y-%m-%d')
nws = nws.sort_values(by=['date'])
nws = nws.query('date >= "2016-01-04"').reset_index(drop=True)
nws = nws[['date','article']].dropna()
nws

CPU times: user 1min 29s, sys: 39.1 s, total: 2min 8s
Wall time: 2min 35s


,date,article
0,2016-01-04,Jan 4 (Reuters) - Here are some upcoming event...
1,2016-01-04,The Verge is back on the ground at CES in Las ...
2,2016-01-04,TEHRAN — When a Saudi state executioner behead...
3,2016-01-04,"Over the past few years, the form factor of th..."
4,2016-01-04,There might be a new Real Housewives of New Yo...
...,...,...
2687308,2020-04-01,"PICTURE PROMPTS Now that schools have closed, ..."
2687309,2020-04-01,The journalists at BuzzFeed News are proud to ...
2687311,2020-04-01,People wait in line outside Trader Joe's in Co...
2687321,2020-04-01,Here's some good news we can all use ... Ruth ...


In [5]:
%%time

"""Агрегируем новости на каждый день, взяв верхние 30 тк учесть все новости у меня не хватает мощностей, затем посчитаем поляризацию
и субьективность новости"""

nws = nws.groupby('date').head(30).reset_index(drop=True)

nws['polarity'] = nws['article'].apply(lambda x: TextBlob(x).sentiment.polarity)
nws['subjectivity'] = nws['article'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
nws = nws[['date','polarity','subjectivity']]
nws

CPU times: user 3min 56s, sys: 7.64 s, total: 4min 3s
Wall time: 4min 7s


,date,polarity,subjectivity
0,2016-01-04,0.220000,0.276667
1,2016-01-04,0.145909,0.386477
2,2016-01-04,-0.005173,0.439542
3,2016-01-04,0.165713,0.459329
4,2016-01-04,0.190569,0.429101
...,...,...,...
46496,2020-04-01,0.180013,0.404167
46497,2020-04-01,0.139510,0.515120
46498,2020-04-01,0.058956,0.436612
46499,2020-04-01,0.203829,0.567793


In [6]:
#Тут группируем по дням и берем среднее значение, как это требуется в статье и джойним два датасета


nws = nws.groupby('date')['polarity','subjectivity'].mean().reset_index()
sp500 = sp500.merge(nws, how = 'left', on =['date'])
sp500

/var/folders/j5/m1xh3jd12k13l381sjc858d40000gn/T/ipykernel_66981/75054216.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  nws = nws.groupby('date')['polarity','subjectivity'].mean().reset_index()


,date,volume,adj_close,polarity,subjectivity
0,2016-01-04,4304880000,2012.66,0.093945,0.441924
1,2016-01-05,3706620000,2016.71,0.090955,0.428697
2,2016-01-06,4336660000,1990.26,0.117847,0.389264
3,2016-01-07,5076590000,1943.09,0.077416,0.447171
4,2016-01-08,4664940000,1922.03,0.070561,0.405114
...,...,...,...,...,...
1022,2020-01-27,3823100000,3243.63,0.059145,0.356702
1023,2020-01-28,3526720000,3276.24,0.065204,0.333752
1024,2020-01-29,3584500000,3273.40,0.112224,0.426403
1025,2020-01-30,3787250000,3283.66,0.096835,0.416740


In [7]:
"""Переведем признаки в стандартное распределение, тк изначально данные неооднородны и разной мастшабности, чтобы 
веса модели учитывались одинаково для всех, а так же в статье нормализируется весь датасет, но это некорректно так делать, потому что
наблюдается в зазор будущее, правильно будет брать с окном, что и было сделано"""

sp500 = sp500.set_index('date')
sp500['target'] = sp500['adj_close'].pct_change()
sp500['target'] = np.where(sp500.target >= 0, 1, 0)

for col in sp500.iloc[:,:-1]:
    roll = sp500[col].rolling(30)
    sp500[col] = (sp500[col] - roll.mean()) / roll.std()
    
sp500 = sp500.dropna()
sp500

,volume,adj_close,polarity,subjectivity,target
date,,,,,
2016-02-16,-0.597277,-0.207687,-0.399156,-2.493325,1
2016-02-17,0.199848,0.602167,0.592586,0.377528,1
2016-02-18,-1.052079,0.542588,-0.622428,-0.216059,0
2016-02-19,-1.627342,0.695789,-2.906649,0.546429,0
2016-02-22,-1.655314,1.590760,1.121159,0.191403,1
...,...,...,...,...,...
2020-01-27,0.342337,-0.137790,-0.624158,-1.142016,0
2020-01-28,-0.002959,0.510046,-0.436063,-1.676942,1
2020-01-29,0.074801,0.392795,0.957640,0.880104,0


## 1. SVM, TextBlob

Оценим модель чисто на новостях, в признаках будем смотреть сентемент анализ за всю прошедшую неделю, когда в статье используются только текущие новости и соотвественно, как я понял, тоже наблюдается зазор в будущее

In [8]:
fund_df = sp500[['polarity', 'subjectivity', 'target']].copy()

for lag in range(1,  5):
    fund_df['lag_pol_{}'.format(lag)] = fund_df['polarity'].shift(lag)
    fund_df['lag_sub_{}'.format(lag)] = fund_df['subjectivity'].shift(lag)

fund_df = fund_df.drop(columns=['polarity', 'subjectivity']).dropna()
fund_df

,target,lag_pol_1,lag_sub_1,lag_pol_2,lag_sub_2,lag_pol_3,lag_sub_3,lag_pol_4,lag_sub_4
date,,,,,,,,,
2016-02-22,1,-2.906649,0.546429,-0.622428,-0.216059,0.592586,0.377528,-0.399156,-2.493325
2016-02-23,0,1.121159,0.191403,-2.906649,0.546429,-0.622428,-0.216059,0.592586,0.377528
2016-02-24,1,-1.271524,-1.012677,1.121159,0.191403,-2.906649,0.546429,-0.622428,-0.216059
2016-02-25,1,1.130939,0.466570,-1.271524,-1.012677,1.121159,0.191403,-2.906649,0.546429
2016-02-26,0,-1.322257,0.176933,1.130939,0.466570,-1.271524,-1.012677,1.121159,0.191403
...,...,...,...,...,...,...,...,...,...
2020-01-27,0,-0.562421,0.727094,-0.941132,-2.190866,0.046252,-0.017925,1.767346,1.631450
2020-01-28,1,-0.624158,-1.142016,-0.562421,0.727094,-0.941132,-2.190866,0.046252,-0.017925
2020-01-29,0,-0.436063,-1.676942,-0.624158,-1.142016,-0.562421,0.727094,-0.941132,-2.190866


In [9]:
features_train, features_valid, target_train, target_valid = train_test_split(
    fund_df.drop('target', axis=1), fund_df.target, shuffle = False , test_size=0.25, random_state=12345)


param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}
 
grid = GridSearchCV(SVC(), param_grid, verbose = 0)
 
grid.fit(features_train, target_train)
pred_valid = grid.predict(features_valid)

print(classification_report(target_valid, pred_valid, zero_division = 1))
print('Confusion matrix')
print(confusion_matrix(target_valid, pred_valid))

              precision    recall  f1-score   support

           0       1.00      0.00      0.00       105
           1       0.58      1.00      0.73       144

    accuracy                           0.58       249
   macro avg       0.79      0.50      0.37       249
weighted avg       0.76      0.58      0.42       249

Confusion matrix
[[  0 105]
 [  0 144]]


## 2. SVM, TextBlob + TA

Тут добавим прошедшие цены закрытия, объемы, и вместо скользящей средней посчитаем средневзвешенную тк она лучше борется с отставанием цены

In [10]:
ta_df = sp500.copy()

def make_features(data, max_lag, news_lag, wma_period):
    
    for lag in range(1, news_lag + 1):
        data['lag_pol_{}'.format(lag)] = data['polarity'].shift(lag)
        data['lag_sub_{}'.format(lag)] = data['subjectivity'].shift(lag)
        
    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['adj_close'].shift(lag)
        data['vol_lag_{}'.format(lag)] = data['volume'].shift(lag)
        
    data['wma'] = data['adj_close'].shift().rolling(wma_period).apply(lambda x: ((np.arange(wma_period)+1)*x).sum()/(np.arange(wma_period)+1).sum())
    

make_features(ta_df, 5, 5, 21)
ta_df = ta_df.drop(columns=['polarity', 'subjectivity', 'adj_close', 'volume']).dropna()
ta_df

,target,lag_pol_1,lag_sub_1,lag_pol_2,lag_sub_2,lag_pol_3,lag_sub_3,lag_pol_4,lag_sub_4,lag_pol_5,...,vol_lag_1,lag_2,vol_lag_2,lag_3,vol_lag_3,lag_4,vol_lag_4,lag_5,vol_lag_5,wma
date,,,,,,,,,,,,,,,,,,,,,
2016-03-16,1,-0.664786,-1.279661,-1.617028,0.322689,1.295594,1.063479,0.718936,0.794552,-0.325645,...,-1.788684,1.575858,-2.064316,1.751193,-1.176109,1.259493,-0.674778,1.346814,-1.357948,1.520567
2016-03-17,1,-0.885695,0.970691,-0.664786,-1.279661,-1.617028,0.322689,1.295594,1.063479,0.718936,...,-0.911862,1.407053,-1.788684,1.575858,-2.064316,1.751193,-1.176109,1.259493,-0.674778,1.534975
2016-03-18,1,0.337832,-0.238232,-0.885695,0.970691,-0.664786,-1.279661,-1.617028,0.322689,1.295594,...,-0.086771,1.483936,-0.911862,1.407053,-1.788684,1.575858,-2.064316,1.751193,-1.176109,1.550304
2016-03-21,1,0.698104,0.308546,0.337832,-0.238232,-0.885695,0.970691,-0.664786,-1.279661,-1.617028,...,2.770379,1.574621,-0.086771,1.483936,-0.911862,1.407053,-1.788684,1.575858,-2.064316,1.562065
2016-03-22,0,0.638636,-0.271626,0.698104,0.308546,0.337832,-0.238232,-0.885695,0.970691,-0.664786,...,-1.680484,1.581687,2.770379,1.574621,-0.086771,1.483936,-0.911862,1.407053,-1.788684,1.561419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-27,0,-0.562421,0.727094,-0.941132,-2.190866,0.046252,-0.017925,1.767346,1.631450,-0.697756,...,0.223190,1.563257,0.309076,1.584295,0.148128,1.679996,0.751662,1.936228,0.283907,1.529231
2020-01-28,1,-0.624158,-1.142016,-0.562421,0.727094,-0.941132,-2.190866,0.046252,-0.017925,1.767346,...,0.342337,0.962379,0.223190,1.563257,0.309076,1.584295,0.148128,1.679996,0.751662,1.373531
2020-01-29,0,-0.436063,-1.676942,-0.624158,-1.142016,-0.562421,0.727094,-0.941132,-2.190866,0.046252,...,-0.002959,-0.137790,0.342337,0.962379,0.223190,1.563257,0.309076,1.584295,0.148128,1.285680


In [11]:
features_train, features_valid, target_train, target_valid = train_test_split(
    ta_df.drop('target', axis=1), ta_df.target, shuffle = False , test_size=0.25, random_state=12345)


param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}
 
grid = GridSearchCV(SVC(), param_grid, verbose = 0)
 
grid.fit(features_train, target_train)
pred_valid = grid.predict(features_valid)

print(classification_report(target_valid, pred_valid, zero_division = 1))
print('Confusion matrix')
print(confusion_matrix(target_valid, pred_valid))

              precision    recall  f1-score   support

           0       1.00      0.00      0.00       103
           1       0.58      1.00      0.73       142

    accuracy                           0.58       245
   macro avg       0.79      0.50      0.37       245
weighted avg       0.76      0.58      0.43       245

Confusion matrix
[[  0 103]
 [  0 142]]
